In [59]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import glob
!pip install openpyxl

day_folder = "/content/drive/My Drive/24 mei/"
file_paths = glob.glob(day_folder + "*.csv")

school_id = 47
class_id = 108

dataframes = [pd.read_csv(file, sep=';') for file in file_paths]
keyfile_general = pd.read_csv("Keyfile_csv.csv")
keyfile_specific = pd.read_excel("keyfile school 47 class 108.xlsx")


file_to_subject = {}
#teacher_tag = '0x24046130BA41'
teacher_tag = '0x24046131EE8C' #different teacher tag on this day

for file in file_paths:
    df_temp = pd.read_csv(file, sep=';')
    tag_id = df_temp['TagId'].iloc[0]  # This is the TrackLabID
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    try:

        subject_id = keyfile_specific.loc[keyfile_specific['tagnummer'] == tagnr, 'subject ID'].values[0]
    except:
        print("Tag not found:" + tag_id)
        print(tagnr)
        subject_id = None

    # Store TrackLabID (tag_id), Tagnumber, and subject ID in the dictionary
    file_to_subject[tag_id] = {'Tagnumber': tagnr, 'subject ID': subject_id}
#print(file_to_subject)

# Filter dataframes to remove unfound tags (except teacher)
filtered_dataframes = [
    df for df in dataframes
    if (file_to_subject.get(df['TagId'].iloc[0]) and
        file_to_subject[df['TagId'].iloc[0]]['subject ID'] is not None) or
       df['TagId'].iloc[0] == teacher_tag
]

dataframes = filtered_dataframes  # Replace original dataframes with filtered ones


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Tag not found:0x24046131F584
15


In [60]:


def process_timestamps(df):
    # Extract 'TrackLabID'
    tag_id = df['TagId'].iloc[0]

    try:
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], format='%m/%d/%Y %H:%M:%S.%f').dt.floor('s')
    except ValueError:
        # Handle cases where 'TimeStamp' is already in datetime format
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp']).dt.floor('s')

    # Keep only the first entry for each second
    df = df.groupby('TimeStamp').first().reset_index()

    for column in ['X', 'Y', 'Z']:
        # Iteratively detects and replaces outliers until no more are found
        while True:
            # Calculates the difference between consecutive values
            diff = df[column].diff()

            # Identify outliers where the difference is greater than 1.5
            outlier_indices = diff[abs(diff) > 1.5].index

            # loop break condition
            if len(outlier_indices) == 0:
                break

            # Replace outliers with the prior value
            for index in outlier_indices:
                if index > 0:  # >0 avoids first row (which has no prior value)
                    df.loc[index, column] = df.loc[index - 1, column]

    # Convert 'TimeStamp' to datetime in all dataframes
    for df_ in dataframes:
        df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%m/%d/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime

    time_range = pd.date_range(
        start=min(df_['TimeStamp'].min() for df_ in dataframes),
        end=max(df_['TimeStamp'].max() for df_ in dataframes),
        freq='s'
    )
    full_df = pd.DataFrame({'TimeStamp': time_range})

    # fills missing seconds
    df = pd.merge(full_df, df, left_on='TimeStamp', right_on='TimeStamp', how='left')

    # Forward fill and then backward fill to handle NaNs at the beginning and end
    df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']].ffill().bfill()
    # Interpolate any remaining NaNs
    df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']].interpolate(method='linear')

    # Add 'TagID' column back to the DataFrame
    df['TagId'] = tag_id
    return df

dataframes = [df for df in [process_timestamps(df) for df in dataframes] if df is not None]



<ipython-input-60-b8737840830b>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%m/%d/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime
<ipython-input-60-b8737840830b>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%m/%d/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime
<ipython-input-60-b8737840830b>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df

In [61]:
import pandas as pd

def find_extremes_and_context(df, column):
    """
    Manual check as outlier detection, was used to determine 1.5 as sensible value for allowed value jumps.
    """

    # Find index of minimum and maximum values
    min_index = df[column].idxmin()
    max_index = df[column].idxmax()

    # Print +-5 values around min/max
    print(f"Minimum value of {column}:")
    print(df.loc[max(0, min_index - 5):min(len(df), min_index + 5), ['TimeStamp', 'X', 'Y', 'Z']])

    print(f"\nMaximum value of {column}:")
    print(df.loc[max(0, max_index - 5):min(len(df), max_index + 5), ['TimeStamp', 'X', 'Y', 'Z']])


for df in dataframes:
    for column in ['X', 'Y']:
        find_extremes_and_context(df, column)

Minimum value of X:
             TimeStamp         X         Y         Z
63 2023-05-24 10:24:15  1.361210  2.630604  0.201736
64 2023-05-24 10:24:16  1.361210  2.630604  0.250655
65 2023-05-24 10:24:17  1.361210  2.630604  0.168428
66 2023-05-24 10:24:18  1.361210  2.630604  0.710234
67 2023-05-24 10:24:19  0.060139  2.476835  0.769282
68 2023-05-24 10:24:20 -0.479717  2.482310  0.671887
69 2023-05-24 10:24:21 -0.479717  2.671704  0.329175
70 2023-05-24 10:24:22 -0.479717  2.671704  0.514568
71 2023-05-24 10:24:23 -0.479717  2.185314  0.609693
72 2023-05-24 10:24:24 -0.479717  2.039225  0.643987
73 2023-05-24 10:24:25 -0.479717  2.009800  0.594491

Maximum value of X:
               TimeStamp         X         Y         Z
3326 2023-05-24 11:18:38  2.579855  2.529258  0.385227
3327 2023-05-24 11:18:39  3.491275  2.529258  0.349135
3328 2023-05-24 11:18:40  4.092721  2.529258  0.290588
3329 2023-05-24 11:18:41  5.060931  2.529258  0.155512
3330 2023-05-24 11:18:42  6.535374  2.529258  0.

In [62]:
#don't actually use the averages, could be used for plotting though (I removed those cells from the notebook because we don't actually need it, could add back if you want)
object_averages = {}

for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    if tag_id == teacher_tag:
        tagnr = '27'
        subject_id = "Teacher"

    elif tag_id in df['TrackLabID'].values:
        tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
        subject_id = keyfile_specific.loc[keyfile_specific['tagnummer'] == tagnr, 'subject ID'].values[0]




    avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].mean()
    object_averages[f"Person_{subject_id} with Tagnr {tagnr}"] = (avg_x, avg_y, avg_z)

avg_df = pd.DataFrame.from_dict(object_averages, orient="index", columns=["X", "Y", "Z"])
print(avg_df)

                                     X         Y         Z
Person_5.0 with Tagnr 28      1.076296  2.530995  0.298459
Person_Teacher with Tagnr 27  7.289628  6.442364  0.404165
Person_4.0 with Tagnr 29      7.112235  3.421582  0.431717
Person_18.0 with Tagnr 34     3.899019  1.642883  0.214871
Person_11.0 with Tagnr 2      1.240954  2.785300  0.171014
Person_1.0 with Tagnr 9       4.025870  3.915144  0.173527
Person_14.0 with Tagnr 18     2.210483  1.967870  0.318467
Person_17.0 with Tagnr 3      1.776840  2.279012  0.317812
Person_13.0 with Tagnr 32     3.605218  3.384001  0.262343
Person_16.0 with Tagnr 1      2.546116  4.121619  0.373227
Person_2.0 with Tagnr 19      6.681565  3.339367  0.370249
Person_8.0 with Tagnr 22      4.426874  1.439592  0.181686
Person_9.0 with Tagnr 5       6.027644  2.465159  0.393569
Person_10.0 with Tagnr 20     1.872741  3.603980  0.178596


In [63]:
fivemin_object_averages = {}
for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # Calculate average for the first 300 rows (= 5 minutes)
    avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].head(300).mean()
    fivemin_object_averages[f"{tagnr}"] = (avg_x, avg_y, avg_z)

avg_df = pd.DataFrame.from_dict(fivemin_object_averages, orient="index", columns=["X", "Y", "Z"])

print(avg_df)

           X         Y         Z
28 -0.056539  2.572238  0.297377
27  4.597380  3.944738  0.230250
29  7.123132  3.418061  0.465075
34  3.994739  2.512773  0.242892
2   1.919684  2.797014  0.232049
9   3.108107  3.923311  0.156911
18  1.938885  1.923275  0.304857
3   1.284276  2.080491  0.250423
32  3.625162  3.534269  0.272305
1   2.636752  4.320339  0.343916
19  6.916002  3.784780  0.357108
22  4.228304  1.450143  0.191531
5   6.205169  2.583869  0.484456
20  1.872741  3.632957  0.191884


In [64]:
#this initially was for baseline distance purposes, decided using avg first 5 minutes might be better though due to trackers potentially not being turned on in usual positions (e.g. turning it on whilst picking it up from teachers desk)
object_start = {}

for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # respective 1st coordinates
    start_x = df_['X'].iloc[0]
    start_y = df_['Y'].iloc[0]
    start_z = df_['Z'].iloc[0]
    object_start[f"{tagnr}"] = (start_x, start_y, start_z)

start_positions_df = pd.DataFrame.from_dict(object_start, orient="index", columns=["X", "Y", "Z"])

print(start_positions_df)

           X         Y         Z
28  1.420882  2.501660  0.259317
27  4.786356  5.679870  0.121700
29  7.047515  3.216540  0.284037
34  3.944577  2.489991  0.201258
2   2.388074  2.778151  0.218346
9   3.515746  3.815850  0.125375
18  1.938885  1.911773  0.054739
3   1.224550  2.042976  0.196361
32  3.599860  3.312277  0.076979
1   2.653262  4.304392  0.576647
19  6.370002  3.167862  0.309657
22  4.371217  1.411913  0.253452
5   6.143755  2.856512  0.379763
20  1.872741  3.550086  0.292102


In [65]:
import numpy as np

def euclidean_distance(x1, y1, x2, y2):
  return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

target_tagnr = 27

# Find target coordinates using tagnr
target_coords = None
for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker == f"{target_tagnr}":
    target_coords = (x, y)
    break




target_x, target_y = target_coords
min_distance = float('inf')
closest_tracker = None

for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker != f"{target_tagnr}":  # Excludes teacher who would have distance 0
    distance = euclidean_distance(target_x, target_y, x, y)
    print(distance, tracker) #can be removed later
    if distance < min_distance:
      min_distance = distance
      closest_tracker = tracker

closest_distance_to_35 = min_distance
print(f"The closest tracker to the teacher is {closest_tracker} with a distance of {closest_distance_to_35}")

4.852083530767245 28
2.580079685824197 29
1.553608358114303 34
2.9133009510184684 2
1.4894271436862265 9
3.3397465465186498 18
3.801588116143384 3
1.0553161104863955 32
1.9962811210194378 1
2.3241333863399753 19
2.5217501806868134 22
2.1064071315101316 5
2.7424197615523425 20
The closest tracker to the teacher is 32 with a distance of 1.0553161104863955


In [66]:
tracker_counts = {}
total_teacher_proximity_count = 0
teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher
print(len(teacher_df))
for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Initialise count for tracker
    tracker_counts[f"{tagnr}"] = 0


    # Iterate through rows using the index
    for index in range(len(df)):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < closest_distance_to_35:
            tracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
sorted_tracker_counts = dict(sorted(tracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within full distance of anyone: {total_teacher_proximity_count}")


exp_amount = 100 / (len(dataframes)-1) #determines expected proximity percentage if it were fully equal(e.g. 20 students = 5%), -1 due to not counting teacher
lower = 0.75
upper = 1.25
for tracker, count in sorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within full distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

7426
Total times teacher was within full distance of anyone: 552
Person 13.0 with Tracker 32: 152 times within full distance of teacher, category: above average with proximity percentage 27.54
Person 18.0 with Tracker 34: 126 times within full distance of teacher, category: above average with proximity percentage 22.83
Person 1.0 with Tracker 9: 98 times within full distance of teacher, category: above average with proximity percentage 17.75
Person 4.0 with Tracker 29: 66 times within full distance of teacher, category: above average with proximity percentage 11.96
Person 16.0 with Tracker 1: 21 times within full distance of teacher, category: below average with proximity percentage 3.80
Person 2.0 with Tracker 19: 19 times within full distance of teacher, category: below average with proximity percentage 3.44
Person 11.0 with Tracker 2: 18 times within full distance of teacher, category: below average with proximity percentage 3.26
Person 9.0 with Tracker 5: 15 times within full dista

In [67]:
half_distance = closest_distance_to_35 / 2
htracker_counts = {}
total_teacher_proximity_count = 0

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    htracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < half_distance:
            htracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
hsorted_tracker_counts = dict(sorted(htracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within half distance of anyone: {total_teacher_proximity_count}")
# Print the sorted counts with subject IDs


for tracker, count in hsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within half distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within half distance of anyone: 83
Person 13.0 with Tracker 32: 49 times within half distance of teacher, category: above average with proximity percentage 59.04
Person 1.0 with Tracker 9: 11 times within half distance of teacher, category: above average with proximity percentage 13.25
Person 18.0 with Tracker 34: 6 times within half distance of teacher, category: average with proximity percentage 7.23
Person 2.0 with Tracker 19: 5 times within half distance of teacher, category: average with proximity percentage 6.02
Person 16.0 with Tracker 1: 4 times within half distance of teacher, category: below average with proximity percentage 4.82
Person 4.0 with Tracker 29: 2 times within half distance of teacher, category: below average with proximity percentage 2.41
Person 9.0 with Tracker 5: 2 times within half distance of teacher, category: below average with proximity percentage 2.41
Person 10.0 with Tracker 20: 2 times within half distance of teacher, category: b

In [68]:
quarter_distance = closest_distance_to_35/4

qtracker_counts = {}
total_teacher_proximity_count = 0

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    qtracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < quarter_distance:
            qtracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
qsorted_tracker_counts = dict(sorted(qtracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within quarter distance of anyone: {total_teacher_proximity_count}")
# Print the sorted counts with subject IDs


for tracker, count in qsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within quarter distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within quarter distance of anyone: 11
Person 13.0 with Tracker 32: 7 times within quarter distance of teacher, category: above average with proximity percentage 63.64
Person 4.0 with Tracker 29: 2 times within quarter distance of teacher, category: above average with proximity percentage 18.18
Person 18.0 with Tracker 34: 1 times within quarter distance of teacher, category: average with proximity percentage 9.09
Person 1.0 with Tracker 9: 1 times within quarter distance of teacher, category: average with proximity percentage 9.09
Person 5.0 with Tracker 28: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 11.0 with Tracker 2: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 14.0 with Tracker 18: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 17.0 with Tracker 3: 0 times within quarter distan

In [70]:

merge_df = pd.read_excel("merged-data-WIP_2025-05-01_17-31 (1) (1) (1).xlsx")  # Replace with your file name

# Create new columns if they don't exist
for distance_type in ['full', 'half', 'quarter']:
    if f'{distance_type}_proximity' not in merge_df.columns:
        merge_df[f'{distance_type}_proximity'] = None
    if f'{distance_type}_category' not in merge_df.columns:
        merge_df[f'{distance_type}_category'] = None


for distance_type, sorted_counts in [('full', sorted_tracker_counts), ('half', hsorted_tracker_counts), ('quarter', qsorted_tracker_counts)]:
    total_count = sum(sorted_counts.values()) # total proximity count for this distance type

    for tracker, count in sorted_counts.items():

        tag_id = df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0]

        # Find the matching row in the existing DataFrame
        mask = (merge_df['school'] == school_id) & (merge_df['class'] == class_id) & (merge_df['tracklab_id'] == tag_id)  # Replace 'tracklab_id' with the actual column name for TagID if different
        matching_row_index = merge_df[mask].index

        # Update the existing DataFrame if matching row is found
        if len(matching_row_index) > 0:
            row_index = matching_row_index[0]
            merge_df.loc[row_index, f'{distance_type}_proximity'] = (count / total_count) * 100

            proximity_percentage = (count / total_count) * 100
            ratio = proximity_percentage / exp_amount

            if ratio < lower:
                category = "below average"
            elif ratio > upper:
                category = "above average"
            else:
                category = "average"

            merge_df.loc[row_index, f'{distance_type}_category'] = category


merge_df.to_excel("merged-data-WIP_2025-05-01_17-31 (1) (1) (1).xlsx", index=False)